In [ ]:
# Test NEW exchanges: Deribit, Bitfinex, Bitmex
TEST_SYMBOL = "BTC"

new_providers = [
    ('Deribit', DeribitOIProvider()),
    ('Bitfinex', BitfinexOIProvider()),
    ('Bitmex', BitmexOIProvider())
]

print(f"\n{'='*70}")
print(f"🆕 NEW EXCHANGES - TESTING {TEST_SYMBOL}")
print(f"{'='*70}\n")

total_new_btc = 0
total_new_usd = 0

for name, provider in new_providers:
    try:
        result = await provider.get_oi_data(TEST_SYMBOL)
        total_new_btc += result.total_oi_tokens
        total_new_usd += result.total_oi_usd
        
        status = "✅" if result.validation_passed else "⚠️"
        print(f"{status} {name:12} | {result.total_oi_tokens:>10,.0f} {TEST_SYMBOL} | ${result.total_oi_usd/1e9:>6.2f}B | {len(result.markets)} markets")
        
        # Show market details
        for market in result.markets:
            print(f"   └─ {market.symbol}: {market.oi_tokens:,.0f} {TEST_SYMBOL} @ ${market.price:,.2f}")
            print(f"      Funding: {market.funding_rate*100:+.4f}% | Market Type: {market.market_type.value}")
        
        print()
        
    except Exception as e:
        print(f"❌ {name}: Error - {e}\n")
    
    finally:
        await provider.close()

print(f"{'-'*70}")
print(f"{'NEW TOTAL':12} | {total_new_btc:>10,.0f} {TEST_SYMBOL} | ${total_new_usd/1e9:>6.2f}B")
print(f"{'='*70}")
print(f"\n✅ NEW EXCHANGES INTEGRATION COMPLETE!")
print(f"📊 Adds ${total_new_usd/1e9:.2f}B to total OI coverage")

# Open Interest (OI) Interactive Prototyping Notebook

## Purpose
This notebook replicates the **exact functionality** of the `/oi <asset>` Telegram command.

Use this notebook to:
- ✅ Test existing exchanges interactively
- ✅ Add new exchanges with minimal code changes
- ✅ Explore OI data across markets
- ✅ Compare exchanges side-by-side
- ✅ Prototype and validate before production

## System Requirements
- **Python**: 3.12.3 (or 3.10+)
- **Location**: `services/market-data/` directory

## Dependencies
```bash
pip install jupyter ipython pandas matplotlib
pip install aiohttp loguru
```

---
## 1. Setup & Imports

In [ ]:
# Standard library
import asyncio
import json
from typing import Dict, List, Optional, Any
from datetime import datetime
from dataclasses import asdict

# Third-party
import pandas as pd
from loguru import logger

# Local OI system modules
from oi_engine_v2 import (
    BaseExchangeOIProvider, 
    MarketOIData, 
    ExchangeOIResult, 
    MarketType,
    AggregatedOIResult
)

# Exchange providers (6 original exchanges)
from binance_oi_provider import BinanceOIProvider
from bybit_oi_provider import BybitOIProvider
from okx_oi_provider import OKXOIProvider
from gateio_oi_provider_working import GateIOOIProviderWorking
from bitget_oi_provider_working import BitgetOIProviderWorking
from hyperliquid_oi_provider import HyperliquidOIProvider

# NEW EXCHANGE PROVIDERS (3 additional exchanges)
from deribit_oi_provider import DeribitOIProvider
from bitfinex_oi_provider import BitfinexOIProvider
from bitmex_oi_provider import BitmexOIProvider

# Unified aggregator (replicates /oi command)
from unified_oi_aggregator import UnifiedOIAggregator, UnifiedOIResponse

print("✅ All modules imported successfully!")
print("📊 Available exchanges: Binance, Bybit, OKX, Gate.io, Bitget, Hyperliquid")
print("🆕 NEW: Deribit, Bitfinex, Bitmex")

---
## 2. Helper Functions for Display

In [ ]:
def display_oi_result(result: ExchangeOIResult, verbose: bool = False):
    """Display OI result in a formatted way"""
    print(f"\n{'='*70}")
    print(f"📊 {result.exchange.upper()} OI ANALYSIS - {result.base_symbol}")
    print(f"{'='*70}")
    
    # Summary
    print(f"\n📈 TOTAL OI:")
    print(f"   Tokens: {result.total_oi_tokens:,.0f} {result.base_symbol}")
    print(f"   USD: ${result.total_oi_usd/1e9:.2f}B")
    print(f"   Markets: {len(result.markets)}")
    print(f"   Volume 24h: ${result.total_volume_24h_usd/1e9:.2f}B")
    
    # Market breakdown
    print(f"\n📊 MARKET BREAKDOWN:")
    for market in result.markets:
        pct = (market.oi_usd / result.total_oi_usd * 100) if result.total_oi_usd > 0 else 0
        print(f"   {market.market_type.value:5} | {market.oi_tokens:10,.0f} {result.base_symbol} | "
              f"${market.oi_usd/1e9:6.2f}B | {pct:5.1f}%")
        
        if verbose:
            print(f"         Symbol: {market.symbol}")
            print(f"         Price: ${market.price:,.2f}")
            print(f"         Funding: {market.funding_rate*100:+.4f}%")
            print(f"         Method: {market.calculation_method}")
    
    # Validation
    if result.validation_passed:
        print(f"\n✅ VALIDATION: PASSED")
    else:
        print(f"\n❌ VALIDATION: FAILED")
        for error in result.validation_errors:
            print(f"   - {error}")

def format_unified_response(response: UnifiedOIResponse) -> str:
    """Format unified response exactly like /oi command output"""
    symbol = response.base_symbol
    data = {
        'exchange_breakdown': response.exchange_breakdown,
        'aggregated_oi': response.aggregated_oi,
        'market_categories': response.market_categories,
        'total_markets': response.total_markets,
        'validation_summary': response.validation_summary
    }
    
    # Extract totals
    total_oi_tokens = data['aggregated_oi'].get('total_tokens', 0)
    total_oi_usd = data['aggregated_oi'].get('total_usd', 0)
    total_markets = data.get('total_markets', 0)
    
    # Market category data
    usdt_data = data['market_categories'].get('usdt_stable', {})
    usdc_data = data['market_categories'].get('usdc_stable', {})
    usd_data = data['market_categories'].get('usd_inverse', {})
    
    usdt_usd = usdt_data.get('total_usd', 0)
    usdc_usd = usdc_data.get('total_usd', 0)
    inverse_usd = usd_data.get('total_usd', 0)
    
    usdt_pct = usdt_data.get('percentage', 0)
    usdc_pct = usdc_data.get('percentage', 0)
    inverse_pct = usd_data.get('percentage', 0)
    
    stablecoin_usd = usdt_usd + usdc_usd
    stablecoin_pct = usdt_pct + usdc_pct
    
    # Build message (exact format from Telegram bot)
    message = f"""📊 OPEN INTEREST ANALYSIS - {symbol}

🔢 MARKET TYPE BREAKDOWN:
• Total OI: {total_oi_tokens:,.0f} {symbol} (${total_oi_usd/1e9:.1f}B)
• Stablecoin-Margined: ${stablecoin_usd/1e9:.1f}B | {stablecoin_pct:.1f}%
  - USDT: ${usdt_usd/1e9:.1f}B ({usdt_pct:.1f}%)
  - USDC: ${usdc_usd/1e9:.1f}B ({usdc_pct:.1f}%)
• Coin-Margined (Inverse): ${inverse_usd/1e9:.1f}B | {inverse_pct:.1f}%
  - USD: ${inverse_usd/1e9:.1f}B ({inverse_pct:.1f}%)

🔢 STABLECOIN MARKETS ({stablecoin_pct:.1f}%): ${stablecoin_usd/1e9:.1f}B
🔢 INVERSE MARKETS ({inverse_pct:.1f}%): ${inverse_usd/1e9:.1f}B
📊 COMBINED TOTAL: ${total_oi_usd/1e9:.1f}B

📈 TOP MARKETS:"""
    
    # Build markets list
    individual_markets = []
    for exchange_data in data['exchange_breakdown']:
        exchange = exchange_data['exchange']
        markets = exchange_data.get('market_breakdown', [])
        for market in markets:
            market_type = market.get('type', 'USDT')
            oi_tokens = market.get('oi_tokens', 0)
            oi_usd = market.get('oi_usd', 0)
            funding = market.get('funding_rate', 0)
            volume_24h = market.get('volume_24h', 0)
            percentage = (oi_usd / total_oi_usd * 100) if total_oi_usd > 0 else 0
            
            category_label = 'STABLE' if market_type in ['USDT', 'USDC'] else 'INVERSE'
            
            individual_markets.append({
                'exchange': exchange.title(),
                'type': market_type,
                'oi_tokens': oi_tokens,
                'oi_usd': oi_usd,
                'percentage': percentage,
                'funding': funding,
                'volume_24h': volume_24h,
                'category_label': category_label
            })
    
    # Sort by OI USD
    individual_markets.sort(key=lambda x: x['oi_usd'], reverse=True)
    
    # Add top markets
    for i, market in enumerate(individual_markets[:13], 1):
        funding_sign = "+" if market['funding'] >= 0 else ""
        volume_formatted = f"{market['volume_24h']/1e6:.0f}M" if market['volume_24h'] >= 1e6 else f"{market['volume_24h']/1e3:.0f}K"
        
        message += f"\n{i}. {market['exchange']} {market['type']}: {market['oi_tokens']:,.0f} {symbol} (${market['oi_usd']/1e9:.1f}B) | {market['percentage']:.1f}% {market['category_label']}"
        message += f"\n   Funding: {funding_sign}{market['funding']*100:.4f}% | Vol: {volume_formatted} {symbol}"
    
    # Summary
    validation = data['validation_summary']
    message += f"""

🏢 COVERAGE SUMMARY:
• Exchanges: {validation.get('successful_exchanges', 0)} working
• Markets: {total_markets} total
• Phase 2A: USDT + USDC support

🚨 MARKET ANALYSIS:
• Sentiment: NEUTRAL ⚪➡️
• Risk Level: NORMAL
• Coverage: Multi-stablecoin across {validation.get('successful_exchanges', 0)} exchanges

🕐 {datetime.now().strftime('%H:%M:%S')} UTC / {(datetime.now().replace(hour=(datetime.now().hour + 8) % 24)).strftime('%H:%M:%S')} SGT"""
    
    return message

def create_comparison_df(results: List[ExchangeOIResult]) -> pd.DataFrame:
    """Create pandas DataFrame for easy comparison"""
    data = []
    for result in results:
        data.append({
            'Exchange': result.exchange.upper(),
            'Total_OI_Tokens': result.total_oi_tokens,
            'Total_OI_USD_B': result.total_oi_usd / 1e9,
            'Markets': len(result.markets),
            'Volume_24h_USD_B': result.total_volume_24h_usd / 1e9,
            'Validated': '✅' if result.validation_passed else '❌'
        })
    return pd.DataFrame(data)

print("✅ Helper functions loaded!")

---
## 3. Test Individual Exchange (Quick Test)

Test a single exchange provider to understand the data structure:

In [ ]:
# Configure test parameters
TEST_SYMBOL = "BTC"  # Change to any symbol: BTC, ETH, SOL, etc.

# Test Binance provider
provider = BinanceOIProvider()

try:
    result = await provider.get_oi_data(TEST_SYMBOL)
    display_oi_result(result, verbose=True)
finally:
    await provider.close()

print(f"\n✅ Single exchange test complete!")

---
## 4. Test All Exchanges (Parallel)

Test all 9 exchanges in parallel and collect results:

**Original 6 exchanges**: Binance, Bybit, OKX, Gate.io, Bitget, Hyperliquid  
**NEW exchanges**: Deribit, Bitfinex, Bitmex

In [ ]:
# Test all exchanges (6 original + 3 new = 9 total)
TEST_SYMBOL = "BTC"

providers = [
    # Original 6 exchanges
    BinanceOIProvider(),
    BybitOIProvider(),
    OKXOIProvider(),
    GateIOOIProviderWorking(),
    BitgetOIProviderWorking(),
    HyperliquidOIProvider(),
    # NEW 3 exchanges
    DeribitOIProvider(),
    BitfinexOIProvider(),
    BitmexOIProvider()
]

async def test_all_exchanges():
    """Test all exchanges in parallel"""
    tasks = [provider.get_oi_data(TEST_SYMBOL) for provider in providers]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    valid_results = []
    for result in results:
        if isinstance(result, Exception):
            print(f"❌ Error: {result}")
        elif isinstance(result, ExchangeOIResult):
            status = "✅" if result.validation_passed else "⚠️"
            valid_results.append(result)
            print(f"{status} {result.exchange.upper()}: ${result.total_oi_usd/1e9:.2f}B")
    
    # Close all providers
    for provider in providers:
        await provider.close()
    
    return valid_results

all_results = await test_all_exchanges()

# Display comparison table
if all_results:
    print(f"\n📊 EXCHANGE COMPARISON:")
    df = create_comparison_df(all_results)
    print(df.to_string(index=False))
    
    total_oi_usd = sum(r.total_oi_usd for r in all_results)
    total_oi_tokens = sum(r.total_oi_tokens for r in all_results)
    print(f"\n💰 TOTAL ACROSS ALL {len(all_results)} EXCHANGES:")
    print(f"   {total_oi_tokens:,.0f} {TEST_SYMBOL} (${total_oi_usd/1e9:.2f}B)")

---
## 5. FULL `/oi` COMMAND REPLICATION

This cell replicates the **EXACT** functionality of `/oi <asset>` command:

In [ ]:
# ⚡ THIS IS THE EXACT /oi COMMAND LOGIC ⚡

SYMBOL = "BTC"  # 🔧 Change this to any symbol: ETH, SOL, etc.

# Create aggregator (same as production)
aggregator = UnifiedOIAggregator()

try:
    print(f"🔍 Analyzing Open Interest for {SYMBOL} across USDT + USDC markets...\n")
    
    # Get unified OI data (same as /oi command)
    unified_response = await aggregator.get_unified_oi_data(SYMBOL)
    
    # Format message (exact same formatting as Telegram bot)
    formatted_message = format_unified_response(unified_response)
    
    # Display result
    print(formatted_message)
    
finally:
    await aggregator.close()

print("\n✅ /oi command replication complete!")

---
## 6. ADD NEW EXCHANGE (Template)

Copy this template to add a new exchange:

In [ ]:
from typing import Optional
import aiohttp

class MyNewExchangeProvider(BaseExchangeOIProvider):
    """
    Template for adding a new exchange
    
    Steps:
    1. Find exchange API documentation for:
       - Open Interest endpoint
       - Ticker/Price endpoint
       - Funding rate endpoint
    2. Implement the 3 required methods below
    3. Test in this notebook!
    """
    
    def __init__(self):
        super().__init__("mynewexchange")
        self.base_url = "https://api.mynewexchange.com"  # 🔧 Change this
    
    def get_supported_market_types(self) -> List[MarketType]:
        """What market types does this exchange support?"""
        return [MarketType.USDT]  # 🔧 Add USDC, USD if available
    
    def format_symbol(self, base_symbol: str, market_type: MarketType) -> str:
        """Format symbol for exchange API"""
        # 🔧 Example formats:
        # - Binance: BTC -> BTCUSDT
        # - Bybit: BTC -> BTCUSDT
        # - OKX: BTC -> BTC-USDT-SWAP
        if market_type == MarketType.USDT:
            return f"{base_symbol}USDT"  # 🔧 Adjust format
        elif market_type == MarketType.USDC:
            return f"{base_symbol}USDC"
        else:
            return f"{base_symbol}USD"
    
    async def get_oi_data(self, base_symbol: str) -> ExchangeOIResult:
        """Fetch OI data from exchange"""
        markets = []
        errors = []
        
        # Fetch each supported market type
        for market_type in self.get_supported_market_types():
            try:
                market_data = await self._fetch_market(base_symbol, market_type)
                if market_data:
                    markets.append(market_data)
            except Exception as e:
                errors.append(f"{market_type.value}: {str(e)}")
        
        # Calculate totals
        total_oi_tokens = sum(m.oi_tokens for m in markets)
        total_oi_usd = sum(m.oi_usd for m in markets)
        total_volume_24h = sum(m.volume_24h for m in markets)
        total_volume_24h_usd = sum(m.volume_24h_usd for m in markets)
        
        # Categorize markets
        usdt_markets = [m for m in markets if m.market_type == MarketType.USDT]
        usdc_markets = [m for m in markets if m.market_type == MarketType.USDC]
        usd_markets = [m for m in markets if m.market_type == MarketType.USD]
        
        return ExchangeOIResult(
            exchange="mynewexchange",
            base_symbol=base_symbol,
            markets=markets,
            total_oi_tokens=total_oi_tokens,
            total_oi_usd=total_oi_usd,
            total_volume_24h=total_volume_24h,
            total_volume_24h_usd=total_volume_24h_usd,
            usdt_markets=usdt_markets,
            usdc_markets=usdc_markets,
            usd_markets=usd_markets,
            validation_passed=len(errors) == 0,
            validation_errors=errors
        )
    
    async def _fetch_market(self, base_symbol: str, market_type: MarketType) -> Optional[MarketOIData]:
        """Fetch single market data from exchange API"""
        try:
            symbol = self.format_symbol(base_symbol, market_type)
            session = await self.get_session()
            
            # 🔧 CUSTOMIZE THESE API CALLS FOR YOUR EXCHANGE
            
            # 1. Fetch Open Interest
            async with session.get(f"{self.base_url}/oi?symbol={symbol}") as response:
                oi_data = await response.json()
                oi_tokens = float(oi_data['openInterest'])  # 🔧 Adjust field name
            
            # 2. Fetch Price & Volume
            async with session.get(f"{self.base_url}/ticker?symbol={symbol}") as response:
                ticker_data = await response.json()
                price = float(ticker_data['lastPrice'])  # 🔧 Adjust field name
                volume_24h = float(ticker_data['volume24h'])  # 🔧 Adjust field name
            
            # 3. Fetch Funding Rate
            async with session.get(f"{self.base_url}/funding?symbol={symbol}") as response:
                funding_data = await response.json()
                funding_rate = float(funding_data['fundingRate'])  # 🔧 Adjust field name
            
            # Calculate USD values
            oi_usd = oi_tokens * price  # For linear contracts
            volume_24h_usd = volume_24h * price
            
            return MarketOIData(
                exchange="mynewexchange",
                symbol=symbol,
                base_symbol=base_symbol,
                market_type=market_type,
                oi_tokens=oi_tokens,
                oi_usd=oi_usd,
                price=price,
                funding_rate=funding_rate,
                volume_24h=volume_24h,
                volume_24h_usd=volume_24h_usd,
                timestamp=datetime.now(),
                api_source=f"API:{symbol}",
                calculation_method=f"linear: {oi_tokens:,.0f} × ${price:,.2f}"
            )
        
        except Exception as e:
            logger.error(f"Error fetching {market_type.value}: {e}")
            return None

print("✅ New exchange template ready!")
print("\n🔧 To use:")
print("   1. Customize the API URLs and field names")
print("   2. Test below:")
print("")
print("# Test your new exchange:")
print("# new_provider = MyNewExchangeProvider()")
print("# result = await new_provider.get_oi_data('BTC')")
print("# display_oi_result(result, verbose=True)")
print("# await new_provider.close()")

---
## 7. Data Exploration & Visualization

Explore OI data with pandas:

In [ ]:
# Create comprehensive DataFrame from all markets
def create_market_df(results: List[ExchangeOIResult]) -> pd.DataFrame:
    """Create detailed market DataFrame"""
    rows = []
    for result in results:
        for market in result.markets:
            rows.append({
                'Exchange': result.exchange.upper(),
                'Symbol': market.symbol,
                'Market_Type': market.market_type.value,
                'OI_Tokens': market.oi_tokens,
                'OI_USD_B': market.oi_usd / 1e9,
                'Price': market.price,
                'Funding_%': market.funding_rate * 100,
                'Volume_24h_USD_B': market.volume_24h_usd / 1e9
            })
    return pd.DataFrame(rows)

# Run full analysis
if 'all_results' in locals() and all_results:
    market_df = create_market_df(all_results)
    
    print("📊 DETAILED MARKET DATA:\n")
    print(market_df.to_string(index=False))
    
    print("\n📈 SUMMARY STATISTICS:\n")
    print(market_df[['OI_USD_B', 'Funding_%', 'Volume_24h_USD_B']].describe())
    
    print("\n💰 OI BY MARKET TYPE:\n")
    print(market_df.groupby('Market_Type')['OI_USD_B'].sum().sort_values(ascending=False))
    
    print("\n🏢 OI BY EXCHANGE:\n")
    print(market_df.groupby('Exchange')['OI_USD_B'].sum().sort_values(ascending=False))
else:
    print("⚠️ Run section 4 first to generate 'all_results' data")

---
## 8. Quick Tests for Different Symbols

Test multiple symbols quickly:

In [ ]:
# Test multiple symbols
SYMBOLS = ["BTC", "ETH", "SOL"]  # 🔧 Add more symbols

aggregator = UnifiedOIAggregator()

try:
    for symbol in SYMBOLS:
        print(f"\n{'='*70}")
        print(f"Testing {symbol}...")
        print(f"{'='*70}")
        
        response = await aggregator.get_unified_oi_data(symbol)
        
        total_oi = response.aggregated_oi['total_usd']
        exchanges = response.validation_summary['successful_exchanges']
        markets = response.total_markets
        
        print(f"✅ {symbol}: ${total_oi/1e9:.2f}B across {exchanges} exchanges ({markets} markets)")
        
finally:
    await aggregator.close()

print("\n✅ Multi-symbol test complete!")

---
## 9. Debugging: Inspect Raw API Responses

Debug individual exchange API responses:

In [ ]:
# Debug: Inspect raw market data
provider = BinanceOIProvider()

try:
    # Test USDT market only
    market_data = await provider._fetch_fapi_market("BTC", MarketType.USDT)
    
    if market_data:
        print("🔍 RAW MARKET DATA INSPECTION:\n")
        print(f"Exchange: {market_data.exchange}")
        print(f"Symbol: {market_data.symbol}")
        print(f"Market Type: {market_data.market_type.value}")
        print(f"OI Tokens: {market_data.oi_tokens:,.2f}")
        print(f"OI USD: ${market_data.oi_usd:,.2f}")
        print(f"Price: ${market_data.price:,.2f}")
        print(f"Funding Rate: {market_data.funding_rate*100:.4f}%")
        print(f"Volume 24h: {market_data.volume_24h:,.2f}")
        print(f"Calculation Method: {market_data.calculation_method}")
        print(f"API Source: {market_data.api_source}")
        print(f"Timestamp: {market_data.timestamp}")
    else:
        print("❌ No data returned")
        
finally:
    await provider.close()

print("\n✅ Raw data inspection complete!")

---
## 10. Save Results to JSON

Export OI data for further analysis:

In [ ]:
---
## Summary

This notebook provides:
- ✅ **Exact `/oi` command replication** (Section 5)
- ✅ **Individual exchange testing** (Section 3)
- ✅ **All exchanges comparison** (Section 4) - **NOW INCLUDES 9 EXCHANGES!**
- ✅ **New exchange template** (Section 6)
- ✅ **Data exploration with pandas** (Section 7)
- ✅ **Multi-symbol testing** (Section 8)
- ✅ **Debugging tools** (Section 9)
- ✅ **Export to JSON** (Section 10)
- 🆕 **NEW: Test Deribit, Bitfinex, Bitmex** (Section 11)

### 🆕 NEW EXCHANGES ADDED:
1. **Deribit**: ~16,000 BTC ($1.74B)
2. **Bitfinex**: ~6,500 BTC ($0.70B)
3. **Bitmex**: ~2,900 BTC ($0.32B)

**Total Coverage**: ~$37.97B BTC Open Interest across **9 exchanges**!

### Next Steps:
1. Run Section 5 to test the full `/oi` command
2. Run Section 11 to test the NEW exchanges
3. Use Section 6 template to add even more exchanges
4. Validate new providers in Section 3
5. Compare with existing exchanges in Section 4
6. Once validated, integrate into `unified_oi_aggregator.py`

---
## 🆕 11. NEW EXCHANGES - Deribit, Bitfinex, Bitmex

Test the three newly integrated exchanges:

---
## Summary

This notebook provides:
- ✅ **Exact `/oi` command replication** (Section 5)
- ✅ **Individual exchange testing** (Section 3)
- ✅ **All exchanges comparison** (Section 4)
- ✅ **New exchange template** (Section 6)
- ✅ **Data exploration with pandas** (Section 7)
- ✅ **Multi-symbol testing** (Section 8)
- ✅ **Debugging tools** (Section 9)
- ✅ **Export to JSON** (Section 10)

### Next Steps:
1. Run Section 5 to test the full `/oi` command
2. Use Section 6 template to add new exchanges
3. Validate new providers in Section 3
4. Compare with existing exchanges in Section 4
5. Once validated, integrate into `unified_oi_aggregator.py`